In [1]:
import torch
print(torch.cuda.device_count())

/scratch4/randalburns/DiffTumor/venv-py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1


In [2]:
%cd STEP1.AutoencoderModel/

/scratch4/randalburns/DiffTumor/STEP1.AutoencoderModel


In [ ]:
import torch
from vq_gan_3d.model.vqgan import VQGAN  # Adjust the import path based on your project structure
from torchvision import transforms
from PIL import Image

# Step 1: Load the pretrained VQGAN model
vqgan_ckpt = '../STEP2.DiffusionModel/pretrained_models/AutoencoderModel.ckpt'  # Replace with the actual path to your checkpoint file
vqgan_model = VQGAN.load_from_checkpoint(vqgan_ckpt).cuda()
vqgan_model.eval()

# Step 2: Define a function to encode images using the encoder from VQGAN
def encode_image(image_path):
    # Define the image transformation pipeline (same as the one used during training)
    transform = transforms.Compose([
        transforms.Resize((128, 128)),  # Adjust based on training preprocessing
        transforms.ToTensor(),
    ])

    # Load and preprocess the image
    image = Image.open(image_path).convert('L')  # Convert to grayscale if necessary
    image = transform(image).unsqueeze(0).cuda()  # Add batch dimension and move to GPU

    with torch.no_grad():
        encoded_image = vqgan_model.encode(image, quantize=False, include_embeddings=True)
        # Normalize the encoded output to match the expected range (-1 to 1)
        encoded_image = ((encoded_image - vqgan_model.codebook.embeddings.min()) /
                         (vqgan_model.codebook.embeddings.max() - vqgan_model.codebook.embeddings.min())) * 2.0 - 1.0
    return encoded_image

# Step 3: Use the function to encode a new CT image
image_path = 'path/to/ct_image.png'  # Replace with the actual path to your CT image
encoded_image = encode_image(image_path)
print(encoded_image.shape)  # Inspect the shape of the encoded image


/scratch4/randalburns/DiffTumor/venv-py38/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/scratch4/randalburns/DiffTumor/venv-py38/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loaded pretrained LPIPS loss from /scratch4/randalburns/DiffTumor/STEP1.AutoencoderModel/vq_gan_3d/model/cache/vgg.pth


/scratch4/randalburns/DiffTumor/venv-py38/lib/python3.8/site-packages/torch/cuda/__init__.py:146: UserWarning: 
NVIDIA A100-PCIE-40GB with CUDA capability sm_80 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA A100-PCIE-40GB GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))
